# Azure Batch サンプル (Bicep + C#)
## Bicep ファイルの作成 (リソースグループ)

In [ ]:
#!pwsh

$bicepFile = @"
targetScope='subscription'

param resourceGroupName string
param resourceGroupLocation string

resource newRG 'Microsoft.Resources/resourceGroups@2023-07-01' = {
  name: resourceGroupName
  location: resourceGroupLocation
}
"@

Out-File -FilePath .\resourceGroup.bicep -InputObject $bicepFile -NoNewline

### Bicep ファイルの作成 (Azure Batch)

In [ ]:
#!pwsh

$bicepFile = @'
targetScope='resourceGroup'
param utcValue string = utcNow()
var randomstring = toLower(replace(uniqueString(subscription().id, resourceGroup().id, utcValue), '-', ''))
@description('Batch Account Name')
var batchAccountName = 'batch${randomstring}'
//param batchAccountName string = 'batch${toLower(uniqueString(resourceGroup().id))}'

@description('Storage Account type')
@allowed([
  'Standard_LRS'
  'Standard_GRS'
  'Standard_ZRS'
  'Premium_LRS'
])
param storageAccountsku string = 'Standard_LRS'

@description('Location for all resources.')
param location string = resourceGroup().location

var storageAccountName = 'storage${randomstring}'
//var storageAccountName = 'storage${uniqueString(resourceGroup().id)}'

resource storageAccount 'Microsoft.Storage/storageAccounts@2023-01-01' = {
  name: storageAccountName
  location: location
  sku: {
    name: storageAccountsku
  }
  kind: 'StorageV2'
  tags: {
    ObjectName: storageAccountName
  }
  properties: {
    minimumTlsVersion: 'TLS1_2'
    allowBlobPublicAccess: false
    networkAcls: {
      defaultAction: 'Allow'
    }
    supportsHttpsTrafficOnly: true
  }
}

resource batchAccount 'Microsoft.Batch/batchAccounts@2024-02-01' = {
  name: batchAccountName
  location: location
  tags: {
    ObjectName: batchAccountName
  }
  properties: {
    allowedAuthenticationModes: [
      'AAD'
      'SharedKey'
      'TaskAuthenticationToken'
    ]
    autoStorage: {
      authenticationMode: 'BatchAccountManagedIdentity'
      storageAccountId: storageAccount.id
    }
  }
  identity : {
    type: 'SystemAssigned'
  }
}

output storageAccountName string = storageAccount.name
output batchAccountName string = batchAccount.name
output location string = location
output resourceGroupName string = resourceGroup().name
output resourceId string = batchAccount.id
'@

Out-File -FilePath .\main.bicep -InputObject $bicepFile -NoNewline

### Azure ログインし Bicepファイルでデプロイを実施

In [ ]:
#!pwsh
$jsonData = (Get-Content ".\bicep-setting.json" | ConvertFrom-Json)
$TENANT_ID = $jsonData.TENANT_ID
$SUBSCRIPTION_GUID = $jsonData.SUBSCRIPTION_GUID
$RESOURCE_GROUP = $jsonData.RESOURCE_GROUP
$LOCATION = $jsonData.LOCATION
$BATCH_POOL_ID = $jsonData.BATCH_POOL_ID
$BATCH_JOB_ID = $jsonData.BATCH_JOB_ID
$BATCH_POOL_NODE_COUNT = $jsonData.BATCH_POOL_NODE_COUNT
$BATCH_POOL_VM_SIZE = $jsonData.BATCH_POOL_VM_SIZE
$SHARED_IMAGE_PUBLISHER = $jsonData.SHARED_IMAGE_PUBLISHER
$SHARED_IMAGE_OFFER = $jsonData.SHARED_IMAGE_OFFER
$SHARED_IMAGE_SKU = $jsonData.SHARED_IMAGE_SKU
$SHARED_IMAGE_VERSION = $jsonData.SHARED_IMAGE_VERSION

#### Azure へログイン

In [ ]:
#!pwsh
az login -t ${TENANT_ID}
az account set --subscription ${SUBSCRIPTOIN_GUID}

#### bicep の upgrade

In [ ]:
#!pwsh
az bicep upgrade

#### リソースグループの作成 (Bicep)

In [ ]:
#!pwsh
az deployment sub create --name demoSubDeployment --location $LOCATION --template-file resourceGroup.bicep --parameters resourceGroupName=$RESOURCE_GROUP resourceGroupLocation=$LOCATION

#### Azure Batch の作成

In [ ]:
#!pwsh
$out = az deployment group create --name batch_create --resource-group $RESOURCE_GROUP --template-file main.bicep --verbose

In [ ]:
#!pwsh
$batchSetting = ($out|ConvertFrom-Json)
$batch_keys = az batch account keys list --name $batchSetting.properties.outputs.batchAccountName.value --resource-group $RESOURCE_GROUP
$_env = @"
TENANT_ID=${TENANT_ID}
LOCATION=${LOCATION}
STORAGE_ACCOUNT_NAME=$($batchSetting.properties.outputs.storageAccountName.value)
BATCH_ACCOUNT_NAME=$($batchSetting.properties.outputs.batchAccountName.value)
BATCH_ACCOUNT_KEY=$(($batch_keys|ConvertFrom-Json).primary)
BATCH_ACCOUNTURL=https://$($batchSetting.properties.outputs.batchAccountName.value).${LOCATION}.batch.azure.com
BATCH_POOL_ID=${BATCH_POOL_ID}
BATCH_JOB_ID=${BATCH_JOB_ID}
BATCH_POOL_NODE_COUNT=${BATCH_POOL_NODE_COUNT}
BATCH_POOL_VM_SIZE=${BATCH_POOL_VM_SIZE}
SHARED_IMAGE_PUBLISHER=${SHARED_IMAGE_PUBLISHER}
SHARED_IMAGE_OFFER=${SHARED_IMAGE_OFFER}
SHARED_IMAGE_SKU=${SHARED_IMAGE_SKU}
SHARED_IMAGE_VERSION=${SHARED_IMAGE_VERSION}
"@
Out-File -FilePath .\.env -InputObject $_env -NoNewline

### Storage Account へのロールアサイン
#### 作成ユーザを所有者(Storage Blob Data Owner)にアサイン

In [ ]:
#!pwsh
$user = (az ad signed-in-user show|ConvertFrom-Json)
az role assignment create --role "Storage Blob Data Owner" --assignee-object-id $user.id --assignee-principal-type User --scope "/subscriptions/${SUBSCRIPTION_GUID}/resourceGroups/${RESOURCE_GROUP}/providers/Microsoft.Storage/storageAccounts/$($batchSetting.properties.outputs.storageAccountName.value)"

#### Azure Batch をBLOBデータ共同作成者(Storage Blob Data Contributor)にアサイン

In [ ]:
#!pwsh
$batch = (az ad sp list --display-name $batchSetting.properties.outputs.batchAccountName.value|ConvertFrom-Json)
az role assignment create --role "Storage Blob Data Contributor" --assignee-object-id $batch.id --assignee-principal-type ServicePrincipal --scope "/subscriptions/${SUBSCRIPTION_GUID}/resourceGroups/${RESOURCE_GROUP}/providers/Microsoft.Storage/storageAccounts/$($batchSetting.properties.outputs.storageAccountName.value)"


## C#(.NET) プログラム実行

### パッケージのインストール

In [ ]:
#r "nuget: Microsoft.Azure.Batch"
#r "nuget: Azure.Storage.Blobs"
#r "nuget: Azure.Identity"
#r "nuget: DotNetEnv, 3.0.0"

### Azure Batch の変数の宣言

In [ ]:
#define NOTEBOOK

using Azure;
using Azure.Core;
using Azure.Storage;
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;
using Azure.Storage.Blobs.Specialized;
using Azure.Storage.Sas;
using Azure.Identity;
using Microsoft.Azure.Batch;
using Microsoft.Azure.Batch.Auth;
using Microsoft.Azure.Batch.Common;
using Microsoft.Extensions.Configuration;
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;


// Update the Batch and Storage account credential strings below with the values unique to your accounts.
// These are used when constructing connection strings for the Batch and Storage client objects.
// Batch account credentials
//Using https://github.com/tonerdo/dotnet-env
DotNetEnv.Env.Load();

readonly string tenantId = DotNetEnv.Env.GetString("TENANT_ID");
readonly string location = DotNetEnv.Env.GetString("LOCATION");
readonly string BatchAccountName = DotNetEnv.Env.GetString("BATCH_ACCOUNT_NAME");
readonly string BatchAccountKey = DotNetEnv.Env.GetString("BATCH_ACCOUNT_KEY");
readonly string BatchAccountUrl = string.Format(DotNetEnv.Env.GetString("BATCH_ACCOUNTURL"), BatchAccountName, location);
// Storage account credentials
readonly string StorageAccountName = DotNetEnv.Env.GetString("STORAGE_ACCOUNT_NAME");
//const string StorageAccountKey = "";
// Batch resource settings
readonly string PoolId = DotNetEnv.Env.GetString("BATCH_POOL_ID");
readonly string JobId = DotNetEnv.Env.GetString("BATCH_JOB_ID");
readonly int PoolNodeCount = DotNetEnv.Env.GetInt("BATCH_POOL_NODE_COUNT");
readonly string PoolVMSize = DotNetEnv.Env.GetString("BATCH_POOL_VM_SIZE");

// Batch Image settings
readonly string publisher = DotNetEnv.Env.GetString("SHARED_IMAGE_PUBLISHER");
readonly string offer = DotNetEnv.Env.GetString("SHARED_IMAGE_OFFER");
readonly string sku = DotNetEnv.Env.GetString("SHARED_IMAGE_SKU");
readonly string version = DotNetEnv.Env.GetString("SHARED_IMAGE_VERSION");



### Blobのクライアントオブジェクト作成

In [ ]:
BlobServiceClient GetBlobServiceClient(string storageAccountName)
{
    BlobServiceClient client = null;
    var blobUri = $"https://{storageAccountName}.blob.core.windows.net";
    try
    {
        client = new BlobServiceClient(new Uri(blobUri), new Azure.Identity.DefaultAzureCredential(
            new Azure.Identity.DefaultAzureCredentialOptions
            {
                // Azure AD のテナント ID を指定する
                AdditionallyAllowedTenants = { "*" },
                TenantId = tenantId
            }
        ));
    }catch(Exception e)
    {
        Console.WriteLine(e);
        throw e;
    }
    return client;
}

### ユーザ委任キーの取得

In [ ]:
async Task<UserDelegationKey> RequestUserDelegationKey(BlobServiceClient blobServiceClient)
{
    // Get a user delegation key for the Blob service that's valid for 1 day
    UserDelegationKey userDelegationKey =
        await blobServiceClient.GetUserDelegationKeyAsync(
            DateTimeOffset.UtcNow,
            DateTimeOffset.UtcNow.AddHours(1));

    return userDelegationKey;
}

### Blobへアップロードするリソースファイルのオブジェクト作成

In [ ]:
ResourceFile UploadFileToContainer(BlobContainerClient containerClient, string containerName, string filePath, string storedPolicyName = null, UserDelegationKey userDelegationKey = null)
{
    Console.WriteLine("Uploading file {0} to container [{1}]...", filePath, containerName);
    string blobName = Path.GetFileName(filePath);
    filePath = Path.Combine(Environment.CurrentDirectory, filePath);

    var blobClient = containerClient.GetBlobClient(blobName);
    Console.WriteLine($"blobName:{blobName} filePath:{filePath}");
    blobClient.Upload(filePath, true);

    // Set the expiry time and permissions for the blob shared access signature. 
    // In this case, no start time is specified, so the shared access signature 
    // becomes valid immediately
    // Check whether this BlobContainerClient object has been authorized with Shared Key.
    if (blobClient.CanGenerateSasUri)
    {
        // Create a SAS token
        var sasBuilder = new BlobSasBuilder()
        {
            BlobContainerName = containerClient.Name,
            BlobName = blobClient.Name,
            Resource = "b"
        };
//
        if (storedPolicyName == null)
        {
            sasBuilder.ExpiresOn = DateTimeOffset.UtcNow.AddHours(1);
            sasBuilder.SetPermissions(BlobContainerSasPermissions.Read);
        }
        else
        {
            sasBuilder.Identifier = storedPolicyName;
        }
//
        var sasUri = blobClient.GenerateSasUri(sasBuilder).ToString();
        // Create a SAS token for the blob resource that's also valid for 1 day
        return ResourceFile.FromUrl(sasUri, filePath);
    }
    else
    {
        BlobSasBuilder sasBuilder = new BlobSasBuilder()
        {
            BlobContainerName = blobClient.BlobContainerName,
            BlobName = blobClient.Name,
            Resource = "b",
            StartsOn = DateTimeOffset.UtcNow,
            ExpiresOn = DateTimeOffset.UtcNow.AddHours(1)
        };
    
        // Specify the necessary permissions
        //sasBuilder.SetPermissions(BlobSasPermissions.Read | BlobSasPermissions.Write);
        sasBuilder.SetPermissions(BlobSasPermissions.Read);
        // Add the SAS token to the blob URI
        BlobUriBuilder uriBuilder = new BlobUriBuilder(blobClient.Uri)
        {
            // Specify the user delegation key
            Sas = sasBuilder.ToSasQueryParameters(
                userDelegationKey,
                containerClient.GetParentBlobServiceClient().AccountName)
        };
        return ResourceFile.FromUrl(uriBuilder.ToUri().ToString(), filePath);
    }
}

### 作成する仮想マシンイメージのオブジェクト作成

In [ ]:
ImageReference CreateImageReference()
{
    return new ImageReference(
        publisher: publisher,
        offer: offer,
        sku: sku,
        version: version);
}

### イメージを元にした仮想マシンの仮想マシンの設定
以下のアドレスで取得可能</br>
https://learn.microsoft.com/en-us/rest/api/batchservice/account/list-supported-images?view=rest-batchservice-2023-11-01&tabs=HTTP

In [ ]:
VirtualMachineConfiguration CreateVirtualMachineConfiguration(ImageReference imageReference)
{
    return new VirtualMachineConfiguration(
        imageReference: imageReference,
        nodeAgentSkuId: "batch.node.windows amd64");
}

### Batch プールの作成

In [ ]:
void CreateBatchPool(BatchClient batchClient, VirtualMachineConfiguration vmConfiguration)
{
    try
    {
        CloudPool pool = batchClient.PoolOperations.CreatePool(
            poolId: PoolId,
            targetDedicatedComputeNodes: PoolNodeCount,
            virtualMachineSize: PoolVMSize,
            virtualMachineConfiguration: vmConfiguration);
        pool.Commit();
    }
    catch (BatchException be)
    {
        // Accept the specific error code PoolExists as that is expected if the pool already exists
        if (be.RequestInformation?.BatchError?.Code == BatchErrorCodeStrings.PoolExists)
        {
            Console.WriteLine("The pool {0} already existed when we tried to create it", PoolId);
        }
        else
        {
            throw; // Any other exception is unexpected
        }
    }
}

### Batch Poolの作成

In [ ]:
#define NOTEBOOK
if (string.IsNullOrEmpty(BatchAccountName) ||
    string.IsNullOrEmpty(BatchAccountKey) ||
    string.IsNullOrEmpty(BatchAccountUrl) ||
    string.IsNullOrEmpty(StorageAccountName))
{
    throw new InvalidOperationException("One or more account credential strings have not been populated. Please ensure that your Batch and Storage account credentials have been specified.");
}
try
{
    Console.WriteLine(BatchAccountName);
    Console.WriteLine(BatchAccountKey);
    Console.WriteLine(BatchAccountUrl);
    Console.WriteLine(StorageAccountName);
    Console.WriteLine("Sample start: {0}", DateTime.Now);
    
    var timer = new Stopwatch();
    timer.Start();
    // Create the blob client, for use in obtaining references to blob storage containers
    var blobServiceClient = GetBlobServiceClient(StorageAccountName);
    // Use the blob client to create the input container in Azure Storage 
    const string inputContainerName = "input";
    var containerClient = blobServiceClient.GetBlobContainerClient(inputContainerName);
    await containerClient.CreateIfNotExistsAsync();
    // The collection of data files that are to be processed by the tasks
    List<string> inputFilePaths = new()
    {
        "taskdata0.txt",
        "taskdata1.txt",
        "taskdata2.txt"
    };
    var userDelegationKey = await RequestUserDelegationKey(blobServiceClient);
    // Upload the data files to Azure Storage. This is the data that will be processed by each of the tasks that are
    // executed on the compute nodes within the pool.
    var inputFiles = new List<ResourceFile>();
    foreach (var filePath in inputFilePaths)
    {
        inputFiles.Add(UploadFileToContainer(containerClient: containerClient, containerName: inputContainerName, filePath: filePath, userDelegationKey: userDelegationKey));
    }
    // Get a Batch client using account creds
    // TODO: BatchAccount Key は セキュリティのためKey Vaultから取り出すよう改造
    // 参考URL https://learn.microsoft.com/ja-jp/azure/batch/batch-aad-auth
    //var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions { ManagedIdentityClientId = BatchManagedIdentityClientId });
    //var credential = new DefaultAzureCredential();
    //AccessToken token = await credential.GetTokenAsync(new Azure.Core.TokenRequestContext(new[] { "https://batch.core.windows.net/" }), new System.Threading.CancellationToken());
    //BatchTokenCredentials cred = new BatchTokenCredentials(BatchAccountUrl, token.Token);
    // 参考URL https://github.com/MicrosoftDocs/azure-docs/issues/109200
    //var tokenCred = new DefaultAzureCredential();
    //using BatchClient batchClient = BatchClient.Open(new BatchTokenCredentials(BatchAccountUrl,
    //async () => (await tokenCred.GetTokenAsync(new TokenRequestContext(new[] { "https://batch.core.windows.net/.default" }))).Token));
    var cred = new BatchSharedKeyCredentials(BatchAccountUrl, BatchAccountName, BatchAccountKey);
    using BatchClient batchClient = BatchClient.Open(cred);
    Console.WriteLine("Creating pool [{0}]...", PoolId);
    // Create a Windows Server image, VM configuration, Batch pool
    ImageReference imageReference = CreateImageReference();
    VirtualMachineConfiguration vmConfiguration = CreateVirtualMachineConfiguration(imageReference);
    CreateBatchPool(batchClient, vmConfiguration);
    // Create a Batch job
    Console.WriteLine("Creating job [{0}]...", JobId);
    try
    {
        CloudJob job = batchClient.JobOperations.CreateJob();
        job.Id = JobId;
        job.PoolInformation = new PoolInformation { PoolId = PoolId };
        job.Commit();
    }
    catch (BatchException be)
    {
        // Accept the specific error code JobExists as that is expected if the job already exists
        if (be.RequestInformation?.BatchError?.Code == BatchErrorCodeStrings.JobExists)
        {
            Console.WriteLine("The job {0} already existed when we tried to create it", JobId);
        }
        else
        {
            throw; // Any other exception is unexpected
        }
    }
    // Create a collection to hold the tasks that we'll be adding to the job
    Console.WriteLine("Adding {0} tasks to job [{1}]...", inputFiles.Count, JobId);
    var tasks = new List<CloudTask>();
    // Create each of the tasks to process one of the input files. 
    foreach (var x in inputFiles.Select((value, index) => new { value, index }))
    {
        string taskId = string.Format("Task{0}", x.index);
        string inputFilename = x.value.FilePath;
        string taskCommandLine = string.Format("cmd /c type {0}", inputFilename);
        var task = new CloudTask(taskId, taskCommandLine)
        {
            ResourceFiles = new List<ResourceFile> { x.value }
        };
        tasks.Add(task);        
    }
    //for (int i = 0; i < inputFiles.Count; i++)
    //{
    //    string taskId = string.Format("Task{0}", i);
    //    string inputFilename = inputFiles[i].FilePath;
    //    string taskCommandLine = string.Format("cmd /c type {0}", inputFilename);
    //    var task = new CloudTask(taskId, taskCommandLine)
    //    {
    //        ResourceFiles = new List<ResourceFile> { inputFiles[i] }
    //    };
    //    tasks.Add(task);
    //}
    // Add all tasks to the job.
    batchClient.JobOperations.AddTask(JobId, tasks);
    // Monitor task success/failure, specifying a maximum amount of time to wait for the tasks to complete.
    TimeSpan timeout = TimeSpan.FromMinutes(30);
    Console.WriteLine("Monitoring all tasks for 'Completed' state, timeout in {0}...", timeout);
    IEnumerable<CloudTask> addedTasks = batchClient.JobOperations.ListTasks(JobId);
    batchClient.Utilities.CreateTaskStateMonitor().WaitAll(addedTasks, TaskState.Completed, timeout);
    Console.WriteLine("All tasks reached state Completed.");
    // Print task output
    Console.WriteLine();
    Console.WriteLine("Printing task output...");
    IEnumerable<CloudTask> completedtasks = batchClient.JobOperations.ListTasks(JobId);
    foreach (CloudTask task in completedtasks)
    {
        string nodeId = string.Format(task.ComputeNodeInformation.ComputeNodeId);
        Console.WriteLine("Task: {0}", task.Id);
        Console.WriteLine("Node: {0}", nodeId);
        Console.WriteLine("Standard out:");
        Console.WriteLine(task.GetNodeFile(Constants.StandardOutFileName).ReadAsString());
    }
    // Print out some timing info
    timer.Stop();
    Console.WriteLine();
    Console.WriteLine("Sample end: {0}", DateTime.Now);
    Console.WriteLine("Elapsed time: {0}", timer.Elapsed);
    // Clean up Storage resources
    await containerClient.DeleteIfExistsAsync();
    Console.WriteLine("Container [{0}] deleted.", inputContainerName);
    // Clean up Batch resources (if the user so chooses)
    Console.WriteLine();
#if NOTEBOOK
    
#if DELETE
    Console.Write($"Delete jobId: {JobId}");
    batchClient.JobOperations.DeleteJob(JobId);
    batchClient.PoolOperations.DeletePool(PoolId);
    Console.Write($"Delete poolId: {PoolId}");
#endif
#else
    Console.Write("Delete job? [yes] no: ");
    string response = Console.ReadLine().ToLower();
    if (response != "n" && response != "no")
    {
        batchClient.JobOperations.DeleteJob(JobId);
    }
    Console.Write("Delete pool? [yes] no: ");
    response = Console.ReadLine().ToLower();
    if (response != "n" && response != "no")
    {
        batchClient.PoolOperations.DeletePool(PoolId);
    }
#endif
}
catch(Exception e) 
{
    Console.WriteLine(e.Message);
}
finally
{ 
#if NOTEBOOK
    Console.WriteLine("Sample complete");
#else
    Console.ReadLine();
    Console.WriteLine("Sample complete, hit ENTER to exit...");
#endif
}